In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split,TimeSeriesSplit
from sklearn import metrics
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from nested_cv import NestedCV

def prepareData(lag):
    # all data
    prec = pd.read_csv('../data/MH25_vaisalawxt520prec_2017.csv')
    wind = pd.read_csv('../data/MH25_vaisalawxt520windpth_2017.csv')
    temp = pd.read_csv('../data/MH30_temperature_rock_2017.csv')
    radio = pd.read_csv('../data/MH15_radiometer__conv_2017.csv')

    # join all data by time
    temp0 = pd.merge(left=temp, right=prec, left_on='time', right_on='time')
    temp0 = pd.merge(left=temp0, right=wind, left_on='time', right_on='time')
    temp0 = pd.merge(left=temp0, right=radio, left_on='time', right_on='time')

    # format season, remove datetime and select columns
    temp0['time'] = pd.to_datetime(temp0['time'])
    temp0['season'] = np.round(pd.DatetimeIndex(temp0['time']).month/3)
    del temp0['time']
    temp0 = temp0[['temperature_5cm [°C]', 'temperature_10cm [°C]',
       'temperature_20cm [°C]', 'temperature_30cm [°C]',
       'temperature_50cm [°C]', 'temperature_100cm [°C]',
       'wind_speed_average [km/h]', 'net_radiation [Wm^-2]', 'season']]

    # difference between ground and deepest temperature --> objective
    diff = abs(temp0['temperature_100cm [°C]'] - temp0['temperature_5cm [°C]'])
    temp0.insert(0, 'delta_t', diff)

    # shift 
    temp0.head()
    temp0['delta_t'] = temp0['delta_t'].shift(lag)
    temp0.head()

    # show correlation (within same time period)
    #print(abs(temp0.corr()['delta_t']).sort_values(ascending=False))

    # clean column names and remove inf/nas
    temp0.columns = temp0.columns.str.replace("[", "_")
    temp0.columns = temp0.columns.str.replace("]", "_")
    temp0 = temp0.fillna(0)
    return temp0


In [135]:

shift = -24
temp0 = prepareData(shift)

# create test and train samples
n = temp0.shape[0]
sep = int(np.round(0.7 * n))
X, y = temp0.iloc[:, 1:], temp0.iloc[:, 0]
y_train = y.iloc[1:sep]
y_test = y.iloc[sep+1:n]
X_train = X.iloc[1:sep,:]
X_test = X.iloc[sep+1:n,:]

# train model and predict
model = sm.OLS(y_train, X_train).fit()
preds = model.predict(X_test)

print(model.summary())

rmse = np.sqrt(metrics.mean_squared_error(y_test, preds))
print("The RMSE is " + str(rmse))

                                 OLS Regression Results                                
Dep. Variable:                delta_t   R-squared (uncentered):                   0.629
Model:                            OLS   Adj. R-squared (uncentered):              0.629
Method:                 Least Squares   F-statistic:                              1156.
Date:                Fri, 29 Nov 2019   Prob (F-statistic):                        0.00
Time:                        19:46:57   Log-Likelihood:                         -11292.
No. Observations:                6131   AIC:                                  2.260e+04
Df Residuals:                    6122   BIC:                                  2.266e+04
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                                coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

In [136]:
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)

rmse = np.sqrt(metrics.mean_squared_error(y_test, preds))
print("The RMSE is " + str(rmse))



The RMSE is 1.7597496937375452


In [138]:

regressor = RandomForestRegressor(n_estimators=20, random_state=0)

tscv = TimeSeriesSplit(n_splits = 12)
rmse = []
for train_index, test_index in tscv.split(X.iloc[:,1]):
    regressor.fit(X.iloc[train_index,:], y.iloc[train_index])
    preds = regressor.predict(X.iloc[test_index,:])
    rmse.append(np.sqrt(metrics.mean_squared_error(y[test_index], preds)))

print(rmse)

[6.367187748059338, 2.2319508003681685, 1.481047705312676, 1.5230658237754773, 2.388319402573476, 1.4199538429012062, 1.173390765630185, 1.2389657487105454, 1.1884814579473617, 1.1862415790729712, 2.1697699806549693, 1.8712862074377075]


In [137]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

rmse = np.sqrt(metrics.mean_squared_error(y_test, preds))
print("The RMSE is " + str(rmse))

The RMSE is 1.804291870782995


/opt/anaconda3/envs/permafrost/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/anaconda3/envs/permafrost/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
